In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import tqdm
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
pd.set_option('display.max_row', 50000)

In [2]:
def doScrollDown(whileSeconds) :
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        dv.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [4]:
###### 검색어
location = input('어디? : ')

###### webdriver 로드
dv = webdriver.Chrome()
dv.get('https://www.naver.com/')
time.sleep(1)

####### 검색시작
search_bar = dv.find_element_by_id('query')
search_bar.send_keys(location)
search_bar.submit()
time.sleep(2)

######## view 페이지
dv.find_element_by_css_selector('#lnb > div.lnb_group > div > ul > li:nth-child(3) > a').click()
time.sleep(1)

######## blog 페이지
dv.find_element_by_css_selector('#snb > div.api_group_option_filter._search_option_simple_wrap > div > div.option_area.type_sort > a:nth-child(2)').click()
time.sleep(1)

######## 스크롤 다운
doScrollDown(15)

######## url ,title 저장
url_list = []
title_list = []
for i in range(1,201):
    text_raw = dv.find_elements_by_css_selector('#sp_blog_'+str(i)+' > div > a.api_txt_lines.total_tit')
    for text in text_raw:
        url=text.get_attribute('href')
        url_list.append(url)
    for text in text_raw:
        title = text.text
        title_list.append(title)
time.sleep(1)
######### 종료
dv.close()

어디? : 대전 여행지


In [5]:
############## 개수
len(url_list),len(title_list)

############### csv로 저장
df = pd.DataFrame({'url':url_list,'title':title_list})
df.to_csv('blog_url.csv',sep=',')

############### csv로 불러오기
url_load = pd.read_csv('blog_url.csv')

In [6]:
url_load.head()

,Unnamed: 0,url,title
0,0,https://blog.naver.com/first_seogu?Redirect=Lo...,겨울 눈 내린 갑천 풍경 물과 눈이 어우러진 언택트 대전 여행지
1,1,https://blog.naver.com/anndam?Redirect=Log&log...,국내 여행지 3월 매화 개화시기 맞춰 대전 동춘당 주말나들이
2,2,https://blog.naver.com/tcacyc?Redirect=Log&log...,국내 여행지 추천 새로운 대전 여행코스 대화동 벽화마을
3,3,https://blog.naver.com/mj9898z?Redirect=Log&lo...,대전 오월드 대전여행지추천
4,4,https://blog.naver.com/smgsl1022?Redirect=Log&...,대전 소제동 맛집 치앙마이방콕 그리고 여행지에서 커피 한잔


In [7]:
number = len(url_list)

In [8]:
############ url당 title,contents 크롤링
############ https://github.com/donghwan2/Crawling-World 참조
content_list = []
for i in tqdm(range(0,number)): 
    url = url_load['url'][i]
    driver = webdriver.Chrome()
    driver.get(url)   
    time.sleep(1)
    try : 
        # iframe 접근
        driver.switch_to.frame('mainFrame')
        # 내용 크롤링
        overlays = ".se-component.se-text.se-l-default"                                 
        contents = driver.find_elements_by_css_selector(overlays)         
        for content in contents:
            content_list.append(content.text)
        content_str = ' '.join(content_list)               
        print(i)
        driver.close()       
    except:
        driver.close()
        print("에러나는 글: " + str(i))
        continue



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
에러나는 글: 86
87
88
89
90
91
92
93
에러나는 글: 94
95
96
97
에러나는 글: 98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
에러나는 글: 117
118
119
120
121
122
123
124
125
126
127
에러나는 글: 128
129
130
131
에러나는 글: 132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
에러나는 글: 172



WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=89.0.4389.128)


In [9]:
# 에러발생 165개만
len(content_str)

183539

In [10]:
f = open('대전여행지.txt','w',encoding='utf8')
f.write(content_str)
f.close()

In [11]:
대전list = pd.DataFrame(content_list)
대전list.to_csv('대전list.csv',encoding='utf8')